In [1]:
# import datetime

def identify_critical_events(data):
    critical_acceleration = 7.5 # 7.5 m/s²
    critical_duration = 2  # number of time steps for 0.2 seconds at 10 Hz
    join_window = 10  # number of time steps for 1 seconds at 10 Hz
    frequency = 10  # 10 Hz

    critical_events = []
    event_start = None
    event_end = None
    event_duration = 0

    for i in range(len(data)):
        # Check if lateral acceleration exceeds the critical acceleration
        if abs(data[i]['lateral_acceleration']) >= critical_acceleration:
            event_duration = event_duration + 1
            if event_duration >= critical_duration:
                event_start = data[i - event_duration + 1]['time']
                event_end = data[i]['time']
                print('start of the event', event_start)
        else:
            if event_start is not None:
                if critical_events != []:
                    if event_start - critical_events[-1][1]  <= join_window/10:
                        print('merge')
                        critical_events[-1] = (critical_events[-1][0], event_end)
                        event_start = None
                        event_end = None
                        event_duration = 0
                    else:
                        print('new event')
                        critical_events.append((event_start, event_end))
                        event_start = None
                        event_end   = None
                        event_duration = 0
                else:
                    print('new event')
                    critical_events.append((event_start, event_end))
                    event_start = None
                    event_end   = None
                    event_duration  = 0

    if event_start is not None: # if the last event is not closed
        critical_events.append((event_start, len(data)/frequency ))

    return critical_events, event_start, event_end

# Example data
start_time = 0.0
time_interval = 0.10
time_stamp = 0

example_data = []
time = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 
        0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 
        1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 
        2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 
        2.6, 2.7, 2.8, 2.9, 3.0]
lateral_accelerations = [0.5, 7.7, 8.0, 8.1, 0.6, 0.5, 7.5, 7.6, 0.4, 0.3, 
                         0.2, 0.8, 0.2, 0.3, 0.1, 0.4, 0.3, 0.2, 0.3, 0.4, 
                         0.5, 0.7, 0.6, 0.5, 0.4, 0.3, 9.2, 9.1, 9.0, 0.3]

example_data = [{'time': time[i], 'lateral_acceleration': lateral_accelerations[i]} for i in range(len(time))]

### Identifying critical events
events, start, end = identify_critical_events(example_data)
print("Critical events (start_index, end_index):", events)

# Optional: Convert indices to time labels
# event_times = [(example_data[start]['time'], example_data[end]['time']) for start, end in events]
# print("Critical event times:", event_times)


start of the event 0.2
start of the event 0.2
new event
start of the event 0.7
merge
start of the event 2.7
start of the event 2.7
new event
Critical events (start_index, end_index): [(0.2, 0.8), (2.7, 2.9)]
